In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AAPL&apikey=98WO4EK4CGOBTPQR&datatype=csv'
df = pd.read_csv(url)
df.head()

,timestamp,open,high,low,close,volume
0,2025-05-12,210.97,211.2679,206.7500,210.79,63775814
1,2025-05-09,199.00,200.5399,197.5350,198.53,36453923
2,2025-05-08,197.72,200.0500,194.6796,197.49,50478872
3,2025-05-07,199.17,199.4400,193.2500,196.25,68616943
4,2025-05-06,198.21,200.6500,197.0200,198.51,51216482
